<a href="https://colab.research.google.com/github/wizard339/article_finder/blob/main/report_metr_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
import pandas as pd
import numpy as np
import re
import zipfile
from collections import namedtuple

In [95]:
data = pd.read_excel('/content/Для анализа.xlsx', index_col='№ п/п')
data.head()

,Дата получения документа,Дата возврата документа,Номер документа,Название изделия/обозначение,"Отдел, разработчик",Количество форматов А4,Фамилия эксперта,"Проект, тема, литера/ примечания",Замечания (шифры),Повторные замечания (шифры)
№ п/п,,,,,,,,,,
1,2022-07-01,2022-07-01,ИТКС.687281.275 СБ,Плата коммутации,"КО, Мартынова",2,Марусина,NaN,NaN,NaN
2,2022-07-01,2022-07-01,ИТКС.687253.075 СБ,Плата многослойная,"КО, Мартынова",4,Марусина,NaN,NaN,NaN
3,2022-07-01,2022-07-01,ИТКС.397-22,ИТКС.461532.006 БИНС-РТ,"ОРИС, Веретенникова",1,Иванов,NaN,NaN,NaN
4,2022-07-01,2022-07-01,ИТКС.461532.006 РО,ИТКС.461532.006 БИНС-РТ,"ОРИС, Веретенникова",2,Иванов,NaN,NaN,NaN
5,2022-07-01,2022-07-01,ИТКС.461532.006-03 РО,ИТКС.461532.006 БИНС-РТ,"ОРИС, Веретенникова",2,Иванов,NaN,NaN,NaN


# 1 Обработка пропусков в данных

In [96]:
# заполняем пропуски в столбце 'Отдел, разработчик' значениями предыдущей строки
data['Отдел, разработчик'] = data['Отдел, разработчик'].fillna(method='ffill')

In [97]:
data['Дата получения документа'] = data['Дата получения документа'].fillna(method='ffill')
data['Дата возврата документа'] = data['Дата возврата документа'].fillna(data['Дата получения документа'])

In [98]:
data[' Замечания (шифры)'] = data[' Замечания (шифры)'].fillna(0)
data['Повторные замечания (шифры)'] = data['Повторные замечания (шифры)'].fillna(0)

# 2 Создание дополнительных столбцов

In [99]:
data['Срок проверки'] = data['Дата возврата документа'] - data['Дата получения документа'] + pd.offsets.Day(1)

In [100]:
data['Разработчик'] = ''

In [101]:
# разделяем значения из столбца 'Отдел, разработчик' на 2 значения
for i in range(data['Отдел, разработчик'].shape[0]):
  data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i].split(',')
  for s in data['Отдел, разработчик'].iloc[i]:
    s = s.strip()
    data['Разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][1].strip()
  data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][0]

<ipython-input-101-65b288692923>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i].split(',')
<ipython-input-101-65b288692923>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][1].strip()
<ipython-input-101-65b288692923>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Отдел, разработчик'].iloc[i]

In [102]:
# исправляем типичные ошибки в названиях отделов + актуализируем названия
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace(['ОРМД', 'ОРМС'],'ОРМ')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace('ОРЛГ', 'ОРИС')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace('ОРСТВГ', 'ОРС ТВГ')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace(['ОРСДВ', 'ОРС ДВ'], 'ОРС ДНГ и ВОГ')

data['Разработчик'] = data['Разработчик'].replace(['Докторова', 'Важнова'], 'Важнова (Докторова)')
data['Фамилия эксперта'] = data['Фамилия эксперта'].replace(['Докторова', 'Важнова'], 'Важнова (Докторова)')
data['Фамилия эксперта'] = data['Фамилия эксперта'].replace(['Мальцева', 'Мацюк'], 'Мацюк (Мальцева)')

In [103]:
# добавляем к фамилии разработчика суффикс с названием его отдела
for i in range(data['Разработчик'].shape[0]):
  data['Разработчик'].iloc[i] = data['Разработчик'].iloc[i] + '_' + data['Отдел, разработчик'].iloc[i]

<ipython-input-103-7ef14eeaf7c5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Разработчик'].iloc[i] = data['Разработчик'].iloc[i] + '_' + data['Отдел, разработчик'].iloc[i]


# 3 Обработка столбца с замечаниями

In [104]:
# для удобства дальнейшей обработки создаем список шифров
for i in range(data[' Замечания (шифры)'].shape[0]):
   a = [s.strip() for s in re.split('[,.]', str(data[' Замечания (шифры)'].iloc[i]))]
   data[' Замечания (шифры)'].iloc[i] = a

for i in range(data['Повторные замечания (шифры)'].shape[0]):
   a = [s.strip() for s in re.split('[,.]', str(data['Повторные замечания (шифры)'].iloc[i]))]
   data['Повторные замечания (шифры)'].iloc[i] = a

<ipython-input-104-916e1521561e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[' Замечания (шифры)'].iloc[i] = a
<ipython-input-104-916e1521561e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Повторные замечания (шифры)'].iloc[i] = a


In [105]:
data.sample(10)

,Дата получения документа,Дата возврата документа,Номер документа,Название изделия/обозначение,"Отдел, разработчик",Количество форматов А4,Фамилия эксперта,"Проект, тема, литера/ примечания",Замечания (шифры),Повторные замечания (шифры),Срок проверки,Разработчик
№ п/п,,,,,,,,,,,,
328,2022-08-15,2022-08-15,ИТКС.687281.217 И,ИТКС.687281.217 Плата коммутации,ОРИС,1,Иванов,присвоение литеры,[0],[0],1 days,Епифанова_ОРИС
123,2022-07-13,2022-07-13,ИТКС.301539.005СБ,Соединитель,КО,1,Важнова (Докторова),NaN,[0],[0],1 days,Авилов_КО
452,2022-09-05,2022-09-06,ИТКС.741124.249,Плита,КО,2,Марусина,NaN,[0],[0],2 days,Хохлов_КО
142,2022-07-13,2022-07-13,ИТКС.713142.001,Стенка,КО,2,Важнова (Докторова),NaN,[0],[0],1 days,Авилов_КО
679,2022-09-26,2022-09-26,ИТКС.43 2/2-22 ПИ,КМИВ.431324.020 БПС-6,КО,1,Марусина,NaN,[0],[0],1 days,Гречушников_КО
476,2022-09-07,2022-09-07,КМИВ.687282.115 СБ,КМИВ.467239.009 БСИ-1,КО,1,Иванов,NaN,[0],[0],1 days,Миннеахметова_КО
226,2022-07-22,2022-07-22,ИТКС.322419.001,ИТКС.323452.020 Контейнер,КО,1,Иванов,присвоение литеры,[0],[0],1 days,Плигин_КО
296,2022-08-08,2022-08-08,ИТКС.794726.013 Е1,АСК БИНС-РС-F ИТКС.794726.013,ОРИС,2,Важнова (Докторова),NaN,[17],[0],1 days,Колесников_ОРИС
139,2022-07-13,2022-07-13,ИТКС.711548.001,Полуниппель,КО,1,Важнова (Докторова),NaN,[0],[0],1 days,Авилов_КО


In [106]:
# функция для подсчета количества ошибок по отделам

def count_codes(data: pd.DataFrame, department: str) -> dict:
  # словарь для хранения количества замечаний по каждому шифру
  codes = dict(zip(np.arange(0, 31), np.zeros(30)))

  # цикл по строкам датафрейма  с помощью указателя i
  for i in range(data.shape[0]):
    # если строка относится к нужному отделу
    if data['Отдел, разработчик'].iloc[i] == department:
      # берем значение из столбца с замечаниями

      for c in data[' Замечания (шифры)'].iloc[i]:
        # добавляем к счетчику шифров
        codes[int(c)] += 1
      for c in data['Повторные замечания (шифры)'].iloc[i]:
        # добавляем к счетчику шифров за исключением 0 шифра
        if int(c) != 0:
          codes[int(c)] += 1

  return codes

In [107]:
# функция для подсчета количества ошибок по разработчику

def count_codes_by_person(data: pd.DataFrame, person: str) -> dict:
  # словарь для хранения количества замечаний по каждому шифру
  codes = dict(zip(np.arange(0, 31), np.zeros(30)))

  # цикл по строкам датафрейма  с помощью указателя i
  for i in range(data.shape[0]):
    # если строка относится к нужному разработчику
    if data['Разработчик'].iloc[i] == person:
      # берем значение из столбца с замечаниями

      for c in data[' Замечания (шифры)'].iloc[i]:
        # добавляем к счетчику шифров
        codes[int(c)] += 1
      for c in data['Повторные замечания (шифры)'].iloc[i]:
        # добавляем к счетчику шифров за исключением 0 шифра
        if int(c) != 0:
          codes[int(c)] += 1

  return codes

# 4 Формирование отчетов

In [108]:
def make_main_report(data: pd.DataFrame) -> pd.DataFrame:
  report = pd.DataFrame(columns=['Проверено документов, шт',
                                   'Проверено листов А4, шт',
                                   'Всего замечаний, шт',
                                   'Доля документов с замечаниями, %',
                                   'Количество замечаний на 100 листов А4, шт'])
  report['Проверено документов, шт'] = data.groupby(['Отдел, разработчик']).count()['Дата получения документа']
  report['Проверено листов А4, шт'] = data.groupby(['Отдел, разработчик']).sum()

  # в цикле создаем вспомогательные переменные для подсчета замечаний и показателей
  for name in report.index.to_list():
    # создаем словарь для каждого отдела с количеством замечаний
    codes = count_codes(data, name)
    # переменная для хранения количества замечаний без учета шифра '17'
    total_without_17 = 0
    # переменная для хранения количества замечаний с шифром '17'
    count_17 = codes[17]
    # переменная для хранения количества документов без замечаний
    without_mistakes = codes[0]
    for c in codes.keys():
      if c not in (0, 17):
        total_without_17 += codes[c]
    # переменная для хранения общего числа замечаний
    total_count = total_without_17 + count_17

    report.loc[name, 'Всего замечаний, шт'] = total_count
    report.loc[name, 'Доля документов с замечаниями, %'] = np.round(((1 - (without_mistakes / report.loc[name, 'Проверено документов, шт'])) * 100), 1)
    report.loc[name, 'Количество замечаний на 100 листов А4, шт'] = np.round(((total_count / report.loc[name, 'Проверено листов А4, шт']) * 100), 1)

  # создаем переменные для хранения сводных показателей по предприятию
  all_docs = np.sum(report['Проверено документов, шт'])
  all_pages = np.sum(report['Проверено листов А4, шт'])
  all_mistakes = np.sum(report['Всего замечаний, шт'])
  all_docs_with_mistakes = np.sum(report['Проверено документов, шт'] * (report['Доля документов с замечаниями, %'] / 100))

  # добавляем в датафрейм строку со сводными показателями
  report.loc['Общая статистика по предприятию'] = [all_docs, all_pages, all_mistakes, np.round((all_docs_with_mistakes / all_docs)*100, 1), np.round((all_mistakes / all_pages)*100, 1)]

  return report

In [109]:
def make_codes_report(data: pd.DataFrame) -> pd.DataFrame:
  Codes = namedtuple('Codes', ['department', 'codes'])
  counts_of_codes = [Codes(dp, count_codes(data, dp)) for dp in data['Отдел, разработчик'].unique()]

  cols = list(data['Отдел, разработчик'].unique())
  cols.insert(0, 'Шифр')
  report = pd.DataFrame(columns=[cols])

  for nt in counts_of_codes:
    report[nt.department] = nt.codes.values()

  report['Шифр'] = report.index

  return report

In [110]:
def make_codes_by_person_report(data: pd.DataFrame) -> pd.DataFrame:
  Codes = namedtuple('Codes', ['person', 'codes'])
  counts_of_codes = [Codes(pers, count_codes_by_person(data, pers)) for pers in data['Разработчик'].unique()]

  cols = list(data['Разработчик'].unique())
  cols.insert(0, 'Шифр')
  report = pd.DataFrame(columns=[cols])

  for nt in counts_of_codes:
    report[nt.person] = nt.codes.values()

  report['Шифр'] = report.index
  report = report.T
  report = report.drop('Шифр', axis=0)

  report.insert(0, 'Отдел', [report.index.values[i][0].split('_')[1] for i in range(report.index.shape[0])])

  return report

In [111]:
def make_time_report(data: pd.DataFrame) -> pd.DataFrame:
  report = pd.DataFrame(data['Срок проверки'].value_counts())
  return report

# 5 Деление датасета на части

In [116]:
# разделим датасет на части по кварталам и полугодиям
data_1q = data[(data['Дата получения документа'] > '2023-01-01') & (data['Дата получения документа'] < '2023-04-01')]
data_2q = data[(data['Дата получения документа'] >= '2023-04-01') & (data['Дата получения документа'] < '2023-07-01')]
data_3q = data[(data['Дата получения документа'] >= '2023-07-01') & (data['Дата получения документа'] < '2023-10-01')]
data_4q = data[(data['Дата получения документа'] >= '2023-10-01') & (data['Дата получения документа'] < '2024-01-01')]
data_1_half_year = data[(data['Дата получения документа'] >= '2023-01-01') & (data['Дата получения документа'] < '2023-07-01')]
data_2_half_year = data[(data['Дата получения документа'] >= '2023-07-01') & (data['Дата получения документа'] < '2024-01-01')]
data_full = data[(data['Дата получения документа'] >= '2023-01-01') & (data['Дата получения документа'] < '2024-01-01')]
# data_list = [data_1q, data_2q, data_3q, data_4q, data_1_half_year, data_2_half_year, data_full]
data_list = [data_3q]

In [117]:
# создадим аттрибуты с именем датасета
data_1q.name = '1_квартал'
data_2q.name = '2_квартал'
data_3q.name = '3_квартал'
data_4q.name = '4_квартал'
data_1_half_year.name = '1_полугодие'
data_2_half_year.name = '2_полугодие'
data_full.name = 'Весь_год'

In [118]:
print(f'1 квартал: {data_1q.shape} \n2 квартал: {data_2q.shape} \n3 квартал: {data_3q.shape}  \n4 квартал: {data_4q.shape} \n1 полугодие: {data_1_half_year.shape}  \n2 полугодие: {data_2_half_year.shape} \nВесь год: {data_full.shape}')

1 квартал: (0, 12) 
2 квартал: (0, 12) 
3 квартал: (722, 12)  
4 квартал: (0, 12) 
1 полугодие: (0, 12)  
2 полугодие: (0, 12) 
Весь год: (0, 12)


# 6 Сохранение отчетов

In [119]:
for d in data_list:
  with zipfile.ZipFile(f'{d.name}.zip', 'w') as zf:
    with zf.open(f'{d.name}_основной_отчет.xlsx', 'w') as buffer:
      with pd.ExcelWriter(buffer) as writer:
        df = make_main_report(d)
        df.to_excel(writer)

    with zf.open(f'{d.name}_основной_по_ошибкам.xlsx', 'w') as buffer:
      with pd.ExcelWriter(buffer) as writer:
        df = make_codes_report(d)
        df.to_excel(writer)

    with zf.open(f'{d.name}_ошибки_по_разработчику.xlsx', 'w') as buffer:
      with pd.ExcelWriter(buffer) as writer:
        df = make_codes_by_person_report(d)
        df.to_excel(writer)

    with zf.open(f'{d.name}_основной_по_срокам.xlsx', 'w') as buffer:
      with pd.ExcelWriter(buffer) as writer:
        df = make_time_report(d)
        df.to_excel(writer)


<ipython-input-108-6751659d5c05>:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  report['Проверено листов А4, шт'] = data.groupby(['Отдел, разработчик']).sum()
<ipython-input-110-46a5a10145bc>:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  report = report.drop('Шифр', axis=0)
